In [ ]:
con = sqlite3.connect(r"D:\Code\Notebooks\SCADA_Bruggen") # zet hier het path naar je eigen database
cur = con.cursor()
df = pd.read_sql("SELECT * FROM WL LIMIT 1000000", con)
con.close()

In [ ]:
df['Tijd'] = pd.to_datetime(df['Tijd'])

In [ ]:
dfday = df[df['Tijd'].dt.date == (np.datetime64('2015-12-12'))]

In [ ]:
uurwindr = dfday[dfday.Omschrijving == "Meteo Actuele Windrichting"]
uurwindr.Waarde = uurwindr.Waarde.astype("float")
dfhourly["WR"] = uurwindr.groupby(uurwindr.Tijd.dt.hour)['Waarde'].mean()

In [ ]:
uurwindk = dfday[dfday.Omschrijving == "Meteo Actuele Windkracht"]
uurwindk.Waarde = uurwindk.Waarde.astype("float")
dfhourly["WK"] = uurwindk.groupby(uurwindk.Tijd.dt.hour)['Waarde'].mean()

In [ ]:
msuur = MeanShift()
scal = StandardScaler()
## Het model trainen om foutmeldingen te voorspellen met windrichting

features = scal.fit_transform(dfhourly[['WK', 'WR']].astype(np.float64))
msmodel.fit(features)

plt.scatter(dfhourly['WR'],  dfhourly['WK'], c= msmodel.labels_, s=50, cmap='viridis')